<a href="https://colab.research.google.com/github/Devi-Prasad-G/image_classification_problem_using_limited_data/blob/main/baseline_with_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow_addons
import tensorflow as tf
import keras
import tensorflow_addons as tfa

## loading trained model and adding dropout layer

In [ ]:
model = tf.keras.models.load_model("/content/drive/MyDrive/adv1/model_best.h5") #best model

In [ ]:
layer_name = "conv5_block3_out"
r = 0.2
model2 = tf.keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
model2.trainable = False #so  that we freeze all weights of trained baseline

inputs = tf.keras.Input(shape=(256, 256, 3))
x = model2(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(r)(x)  # dropout with r probability
outputs = tf.keras.layers.Dense(1000,activation = "softmax",kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(x)
model_with_dropout = tf.keras.Model(inputs, outputs)
model_with_dropout.summary()

## Loading train data

In [ ]:
!unzip /content/drive/MyDrive/train.zip -d /content # to unzip the training file in drive

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(horizontal_flip=True,
                                  fill_mode='nearest',
                                   validation_split=0.2)
training_set = train_datagen.flow_from_directory('./train',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='training')

validation_set = train_datagen.flow_from_directory('./train',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',subset='validation')

## model parameters and savepoints

In [ ]:
l_rate = 0.01
wd = 0.001
loss = tf.keras.losses.CategoricalCrossentropy()
optim = tfa.optimizers.SGDW(learning_rate=l_rate, weight_decay=wd, momentum=0.9, nesterov = False, name = 'SGDW')
#optim = tf.keras.optimizers.Adam()

In [ ]:
model_with_dropout.compile(loss=loss,
              optimizer=optim,
              metrics=['accuracy'])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/adv1/model_test_1.h5",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

## Training the model

In [ ]:
model_with_dropout.fit(training_set, validation_data = validation_set, epochs=20,steps_per_epoch=training_set.samples//32,validation_steps=validation_set.samples //32,verbose=1,callbacks = [checkpoint])

## Fine-tuning the model

In [ ]:
model2.trainable = True
model_with_dropout.summary()

loss = tf.keras.losses.CategoricalCrossentropy()
optim = tf.keras.optimizers.Adam(1e-5) # adding very low learning rate for fine tuning purposes

model_with_dropout.compile(loss=loss,
              optimizer=optim,
              metrics=['accuracy'])

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/adv1/model_test1_finetuning.h5",
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model_with_dropout.fit(training_set_cropped, validation_data = validation_set_cropped, epochs=10,steps_per_epoch=training_set.samples//32,validation_steps=validation_set.samples //32,verbose=1,callbacks = [checkpoint])

In [ ]:
model_with_dropout.save("/content/drive/MyDrive/adv/model_test1_final.h5")